In [1]:
import requests
import json
import lendingclub.account_info as acc_info
import re
from sklearn.externals import joblib
import lendingclub.dataprep_and_modeling.modeling_utils.data_prep_new as data_prep
import lendingclub.investing.investing_utils as investing_utils

# constants
token = acc_info.token
header = {
    'Authorization': token,
    'Content-Type': 'application/json',
    'X-LC-LISTING-VERSION': '1.2'
}
min_score = -0.035764345824470828

In [3]:
# platform = 'lendingclub'

# store = pd.HDFStore(
#     '/Users/justinhsi/justin_tinkering/data_science/lendingclub/{0}_store.h5'.
#     format(platform),
#     append=True)

# test_loans = store['test_filtered_columns']
# train_loans = store['train_filtered_columns']
# store.close()

In [4]:
reload(investing_utils)

<module 'lendingclub.investing.investing_utils' from '/Users/justinhsi/justin_tinkering/data_science/lendingclub/investing/investing_utils.pyc'>

In [5]:
api_loans, api_ids = investing_utils.get_loans_and_ids(header, True)
api_loans = investing_utils.match_col_names(api_loans)
api_loans = investing_utils.match_existing_cols_to_csv(api_loans)
api_loans = investing_utils.make_missing_cols_and_del_dates(api_loans)

In [6]:
len(api_loans)

223

In [7]:
# if investing_utils.verify_df_base_cols(api_loans, test_loans):
#     print('good to go')
#     test_loans = test_loans[api_loans.columns]

In [8]:
# examine_api_pre_standardized = data_prep.prep_data_cols(api_loans)
# examine_test_pre_standardized = data_prep.prep_data_cols(test_loans)
# examine_test, _ = data_prep.process_data_test(test_loans)
# for col in examine_test.columns.values:
#     if len(examine_test[examine_test[col].isnull()]) > 0:
#         print(col)
# for col in examine_api.columns.values:
#     if len(examine_api[examine_api[col].isnull()]) > 0:
#         print(col)
api_X, _ = data_prep.process_data_test(api_loans)

In [9]:
# score the api_loans
regr = joblib.load(data_prep.base_path + '/model_dump/model_0.2.1.pkl')
api_yhat = regr.predict(api_X)

In [10]:
regr.predict(api_X.loc[0,:])

/Users/justinhsi/justin_tinkering/justin_tinker/lib/python2.7/site-packages/sklearn/utils/validation.py:395: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)


array([-0.083301])

In [11]:
api_ids[api_ids == 116999298]

Series([], Name: id, dtype: int64)

In [12]:
len(api_X)

223

In [13]:
ids_and_scores = pd.Series(dict(zip(api_ids,api_yhat)))

In [14]:
ids_and_scores.describe()

count    223.000000
mean      -0.216885
std        0.089152
min       -0.504869
25%       -0.284159
50%       -0.204314
75%       -0.148518
max       -0.057372
dtype: float64

In [15]:
to_pick_from = ids_and_scores[ids_and_scores >= -.07]

In [16]:
len(ids_and_scores)

223

In [17]:
len(to_pick_from)

6

In [18]:
to_pick_from

117628923   -0.061541
117686670   -0.069319
118097554   -0.064102
118257837   -0.067281
118310857   -0.057372
118329475   -0.062893
dtype: float64

In [19]:
to_pick_from.sort_values(ascending=False)

118310857   -0.057372
117628923   -0.061541
118329475   -0.062893
118097554   -0.064102
118257837   -0.067281
117686670   -0.069319
dtype: float64

In [21]:
portfolios = json.loads(
    requests.get(
        'https://api.lendingclub.com/api/investor/v1/accounts/40375293/portfolios',
        headers=header).content)

In [22]:
portfolios

{u'myPortfolios': [{u'portfolioDescription': u'rf regressor 0.2.1 8-27-2017',
   u'portfolioId': 121622034,
   u'portfolioName': u'0.2.1_rf'},
  {u'portfolioDescription': None,
   u'portfolioId': 121452018,
   u'portfolioName': u'pre_justin_RF'}]}

In [43]:
# checking to see if I can get loan ids of owned notes

In [47]:
my_notes = json.loads(requests.get(
        'https://api.lendingclub.com/api/investor/v1/accounts/40375293/notes',
        headers=header).content)

In [51]:
examine_notes = pd.DataFrame(my_notes['myNotes'])

In [54]:
examine_notes['loanStatus'].value_counts(dropna=False)

Current               1695
Fully Paid            1079
Charged Off            184
Issued                 124
Late (31-120 days)      53
In Grace Period         26
Late (16-30 days)       13
Default                  4
Name: loanStatus, dtype: int64

In [58]:
examine_notes[examine_notes['loanStatus'] == 'Issued']['issueDate'].max()

u'2017-08-23T09:19:19.000-07:00'

In [128]:
investing_utils.get_loans_and_ids(header, False)[0].shape

(198, 118)

In [129]:
investing_utils.get_loans_and_ids(header, True)[0].shape

(198, 118)

In [126]:
reload(investing_utils)

<module 'lendingclub.investing.investing_utils' from '/Users/justinhsi/justin_tinkering/data_science/lendingclub/investing/investing_utils.py'>

# Below is testing actual submission of orders

In [22]:
summary_dict = json.loads(
    requests.get(
        'https://api.lendingclub.com/api/investor/v1/accounts/40375293/summary',
        headers=header).content)
cash_to_invest = summary_dict['availableCash']
cash_to_invest

465.4

In [2]:
orders_payload = json.dumps({'aid': 40375293,
                             'orders':[{'loanId': 118310857,
                                        'requestedAmount': float(25.0),
                                        'portfolioId': 121622034}]})

In [3]:
order_response = requests.post(
    'https://api.lendingclub.com/api/investor/v1/accounts/40375293/orders',
headers=header, data=orders_payload)

In [4]:
order_response.content

'{"orderInstructId":166150773,"orderConfirmations":[{"loanId":118310857,"requestedAmount":25.0,"investedAmount":25.0,"executionStatus":["ORDER_FULFILLED","NOTE_ADDED_TO_PORTFOLIO"]}]}'

In [4]:
#118310857 
orders_payload = json.dumps({'aid': 40375293,
                             'orders':[{'loanId': 116999298,
                                        'requestedAmount': float(25.0),
                                        'portfolioId': 121622034}]})

In [7]:
print(orders_payload)

{"aid": 40375293, "orders": [{"loanId": 116999298, "portfolioId": 121622034, "requestedAmount": 25.0}]}


In [9]:
%run invest_script_instant.py

will pause 2 seconds
right now it is 12:03:35, pausing until 12:03:37
Ran investment round.
Cash to invest: $1286.46, meaning 51 possible notes to invest in at $25.0 each.
219 loans seen through api, of which 0 passed min_score of -0.0459971427699
Max scores from this batch was: 118329475   -0.062893
118097554   -0.064102
118257837   -0.067281
117686670   -0.069319
118306680   -0.070137
dtype: float64
Below is from response
(<Response [200]>, '{"orderInstructId":null}')
reached end of invest_script.py


# trying it via script

In [33]:
%run invest_script.py

In [41]:
import time

In [47]:
time.strftime('%Y-%m-%d:%H-%M-%S')

'2017-08-28:15-54-42'

In [14]:
cash_to_invest

490.4

In [33]:
import ConfigParser

In [35]:
cfg_parser = ConfigParser.ConfigParser()

In [36]:
cfg_parser.optionxform?

In [86]:
%run count_loans.py

2017-08-29:01-03-20.763
https://api.lendingclub.com/api/investor/v1/loans/listing?filterId=106510869&showAll=true
395
2017-08-29:01-03-21.825


In [57]:
import pause, datetime

In [84]:
datetime.datetime.now().strfmt()

AttributeError: 'datetime.datetime' object has no attribute 'strfmt'

In [81]:
# pause 3 seconds, then print hello world
now = datetime.datetime.now()
current_hour = now.hour
pause_until = datetime.datetime(now.year, now.month, now.day, now.hour,
                                now.minute, now.second + 3)
pause.until(pause_until)
print('hello world')

hello world
